In [1]:
with open('wordlist_solutions.txt') as f:
    lines = f.readlines()
    
print("Example: ROATE 🟨⬛🟩⬛⬛\n         CRAZY ⬛🟨🟩⬛⬛")
A = input("The letters in the word and in the correct spot (Ex: ??a??): ")
B = input("The letters is in the word but in the wrong spot (Ex: r1 r2): ")
C = input("The letters is not in the word in any spot (Ex: oteczy): ")
print("")

B_char = []
B_pos = []
for i in B:
    if 'a' <= i <= 'z':
        B_char.append(i)
    elif '0' <= i <= '9':
        B_pos.append(int(i))
        
num = 0
for i in lines:
    flag = 1                          ## 1: possible, -1: impossible
    if A[0] != '?' and A[0] != i[0]:
        continue
    if A[1] != '?' and A[1] != i[1]:
        continue
    if A[2] != '?' and A[2] != i[2]:
        continue
    if A[3] != '?' and A[3] != i[3]:
        continue
    if A[4] != '?' and A[4] != i[4]:
        continue
        
    for j in range(len(B_char)):
        if B_char[j] not in i:
            flag = -1
            break
        elif B_char[j] == i[B_pos[j]-1]:
            flag = -1
            break
            
    if flag == 1:
        for j in C:
            if j in i:
                flag = -1
                break
                
    if flag == 1:
        print(i, end='')
        num += 1
        
print("# of possible solutions :", num)

Example: ROATE 🟨⬛🟩⬛⬛
         CRAZY ⬛🟨🟩⬛⬛
The letters in the word and in the correct spot (Ex: ??a??): ??a??
The letters is in the word but in the wrong spot (Ex: r1 r2): r1 r2
The letters is not in the word in any spot (Ex: oteczy): oteczy

dwarf
shard
flair
snarl
wharf
award
swarm
quark
sharp
shark
alarm
spark
guard
# of possible solutions : 13
